Go ahead and install all the required packages using pip installer

Now lets import all the packages 

In [6]:
import streamlit as st
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import json

2. make sure you load the csv file using pandas

In [3]:
df = pd.read_csv("factors_of_women.csv")
df.head()

,S.NO,STATES,YEAR,RAPE,KIDNAP & ABDUCTION,DOWRY DEATHS,CRUETLY BY HUSBAND OR HIS RELATIVES,HUMAN TRAFFICKING,ASSAULT ON MODESTY,SEXUAL HARRASSMENT,others,tot
0,1,Andhra Pradesh,2011,1442,1612,599,13376,0,0,3658,497,28246
1,2,Arunachal Pradesh,2011,42,60,0,18,0,0,0,0,171
2,3,Assam,2011,1700,3192,121,5246,2,0,8,21,11503
3,4,Bihar,2011,934,3050,1413,2607,10,0,11,23,10231
4,5,Chhattisgarh,2011,1053,365,104,834,2,0,174,15,4219


In [16]:
indian_states = json.load(open('india_states.geojson', 'r'))

state_id_map = {}
for feature in indian_states['features']:
    feature['id'] = feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']] = feature['id']
    
df['id'] = df['STATES'].apply(lambda x : state_id_map[x])

In [25]:
df[df["YEAR"] == 2019]

,S.NO,STATES,YEAR,RAPE,KIDNAP & ABDUCTION,DOWRY DEATHS,CRUETLY BY HUSBAND OR HIS RELATIVES,HUMAN TRAFFICKING,ASSAULT ON MODESTY,SEXUAL HARRASSMENT,others,tot
288,1,Andhra Pradesh,2019,1086,589,112,7851,69,4444,1892,88,17746
289,2,Arunachal Pradesh,2019,63,68,1,54,1,80,6,0,317
290,3,Assam,2019,1773,6989,156,11943,153,4619,385,12,30025
291,4,Bihar,2019,730,9025,1120,2397,1,312,11,40,18587
292,5,Chhattisgarh,2019,1036,2033,76,732,25,1316,232,12,7689
293,6,Goa,2019,72,52,1,9,34,119,37,1,329
294,7,Gujarat,2019,528,922,9,3619,8,1048,16,24,8799
295,8,Haryana,2019,1480,2803,248,4867,8,2581,173,28,14683
296,9,Himachal Pradesh,2019,359,337,4,228,0,498,72,10,1636
297,10,Jammu & Kashmir,2019,223,893,8,347,0,1440,15,0,3069
